# 泰坦尼克号生存预测——模型选择与训练

LIJIANcoder97

##### 上一次作业已经将训练集的数据清理完毕了本节将继续清理测试集的数据并使用 KNN，逻辑回归，支持向量机模型进行预测，本文主要研究KNN预测

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

#### 调用sklearn LabelEncoder()，进行编码，文本转数字。

In [2]:
def encode_features(data_set, feature_names):
    for feature_name in feature_names:
        le = LabelEncoder()
        le.fit(data_set[feature_name])
        encoded_column = le.transform(data_set[feature_name])
        data_set[feature_name] = encoded_column
    return data_set

In [3]:
Taitan_train=pd.read_csv('dataset/titanic/train.csv')
Taitan_test=pd.read_csv('dataset/titanic/test.csv')

丢弃'Cabin','Name','Ticket'属性列

In [4]:
Taitan_train=Taitan_train.drop(['Cabin','Name','Ticket'],axis=1)
Taitan_test=Taitan_test.drop(['Cabin','Name','Ticket'],axis=1)

#### 对sex embark属性进行编码

In [5]:
le = LabelEncoder()
le.fit(Taitan_train['Sex'])
Taitan_train['Sex']=le.transform(Taitan_train['Sex']) 

In [6]:
le = LabelEncoder()
le.fit(Taitan_test['Sex'])
Taitan_test['Sex']=le.transform(Taitan_test['Sex']) 

#### Embarked 属性有缺失值，要先进行处理，缺失值替换为最多的登船口

In [7]:
Taitan_train['Embarked'] = Taitan_train['Embarked'].fillna('S')

In [8]:
l = LabelEncoder()
l.fit(Taitan_train['Embarked'])
l.classes_

array(['C', 'Q', 'S'], dtype=object)

In [9]:
Taitan_train['Embarked']=l.transform(Taitan_train['Embarked']) 

In [10]:
Taitan_test['Embarked'] = Taitan_test['Embarked'].fillna('S')
l = LabelEncoder()
l.fit(Taitan_test['Embarked'])
Taitan_test['Embarked']=l.transform(Taitan_test['Embarked']) 

#### 编码完毕
### 对Age 属性进行处理，缺失值替换

In [11]:
Taitan_train['Age'] = Taitan_train['Age'].fillna(Taitan_train['Age'].median())

In [12]:
Taitan_test['Age'] = Taitan_test['Age'].fillna(Taitan_test['Age'].median())

#### 创建family属性代表乘客的亲人数目探索其与存活率的关系

In [13]:
Taitan_train['Family']=Taitan_train['SibSp']+Taitan_train['Parch']+1

In [14]:
Taitan_test['Family']=Taitan_test['SibSp']+Taitan_test['Parch']+1

现在已经有了清理好的数据集了

In [15]:
Taitan_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family
0,1,0,3,1,22.0,1,0,7.2500,2,2
1,2,1,1,0,38.0,1,0,71.2833,0,2
2,3,1,3,0,26.0,0,0,7.9250,2,1
3,4,1,1,0,35.0,1,0,53.1000,2,2
4,5,0,3,1,35.0,0,0,8.0500,2,1


In [41]:
Taitan_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Family
0,892,3,1,34.5,0,0,7.8292,1,1
1,893,3,0,47.0,1,0,7.0000,2,2
2,894,2,1,62.0,0,0,9.6875,1,1
3,895,3,1,27.0,0,0,8.6625,2,1
4,896,3,0,22.0,1,1,12.2875,2,3


In [17]:
Taitan_p=Taitan_train['Survived']
Taitan_train=Taitan_train.drop(['Survived'],axis=1)

# 模型训练
## KNN

In [19]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

导入knn和准确率评估方法

首先通过网格搜索选出最合适的超参数，将训练集分为两部分一部分用于训练，一部分用于验证。

In [20]:
x_train=Taitan_train[0:712]
x_test=Taitan_train[712: ]
y_train=Taitan_p[0:712]
y_test=Taitan_p[712: ]

训练集分割完毕开始训练,使用网格搜索

In [29]:
for k in [2,3,4,5,6]:
    for p in [1,2]: ##1代表明可夫斯基距离，2代表欧拉距离
        for w in ['uniform','distance']: ##uniform 代表不考虑距离权重
            n= KNeighborsClassifier(n_neighbors=k,weights=w,p=p)
            n.fit(x_train,y_train)##构造模型！！
            y_p=n.predict(x_test)##预测
            print('k=',k,';p=',p,';w=',w,';结果:',classification_report(y_test,y_p))

k= 2 ;p= 1 ;w= uniform ;结果:               precision    recall  f1-score   support

           0       0.70      0.91      0.79       115
           1       0.64      0.28      0.39        64

   micro avg       0.69      0.69      0.69       179
   macro avg       0.67      0.60      0.59       179
weighted avg       0.68      0.69      0.65       179

k= 2 ;p= 1 ;w= distance ;结果:               precision    recall  f1-score   support

           0       0.73      0.72      0.73       115
           1       0.52      0.53      0.52        64

   micro avg       0.65      0.65      0.65       179
   macro avg       0.62      0.63      0.63       179
weighted avg       0.66      0.65      0.65       179

k= 2 ;p= 2 ;w= uniform ;结果:               precision    recall  f1-score   support

           0       0.65      0.95      0.77       115
           1       0.50      0.09      0.16        64

   micro avg       0.64      0.64      0.64       179
   macro avg       0.58      0.52      0.47

准确率的参考属性时micro

准确率最高为 0.72，分别是 3，1，uniform；3，1，distance；4，1，distance
选择4，1，distance

In [33]:
n= KNeighborsClassifier(n_neighbors=4,weights='distance',p=1)
n.fit(Taitan_train,Taitan_p)##构造模型！！
y_re=n.predict(Taitan_test)##预测

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [37]:
Taitan_test.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           1
Embarked       0
Family         0
dtype: int64

### 测试集竟然由空值，一定要注意

In [43]:
Taitan_test['Fare'] = Taitan_train['Fare'].fillna(Taitan_train['Fare'].median())

In [44]:
Taitan_test.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
Family         0
dtype: int64

清理完毕再次测试

In [45]:
n= KNeighborsClassifier(n_neighbors=4,weights='distance',p=1)
n.fit(Taitan_train,Taitan_p)##构造模型！！
y_re=n.predict(Taitan_test)##预测

In [47]:
y_re

array([0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0,

In [55]:
Taitan_test['Survived']=y_re

In [61]:
Taitan_re=Taitan_test[['PassengerId','Survived']]

In [62]:
Taitan_re.to_csv('dataset/titanic/RE.csv')

In [63]:
Taitan_re.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,1
4,896,0


In [64]:
Taitan_re.describe()

,PassengerId,Survived
count,418.000000,418.000000
mean,1100.500000,0.351675
std,120.810458,0.478065
min,892.000000,0.000000
25%,996.250000,0.000000
50%,1100.500000,0.000000
75%,1204.750000,1.000000
max,1309.000000,1.000000


# 分析

knn的训练与预测完成了，理想情况下，测试集的预测结果准确率也会达到0.72


上方我们用网格搜索的方式用训练集找到了最合适的超参数，从输出结果可以看出准确率在0.43到0.72件变化，超参数对模型训练结果形像巨大。上方我们超参数的组合共有20种组合，而k和p还可以取更多的值，如果人工调节非常浪费时间，用网格搜索方式可节省时间。